In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from confusion_matrix import CM
import os

import warnings
warnings.filterwarnings("ignore")

from keras import models, optimizers, layers
from keras import backend as K

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding,TimeDistributed,Conv2D,Input, Flatten,Reshape ,MaxPooling2D, Dropout,Activation, LSTM,  Concatenate, Multiply, Lambda
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from capsule_layer import CapsuleLayer, Length, Mask, margin_loss, squash

import argparse
from keras import callbacks
from keras import initializers, layers
from keras.utils import to_categorical
import time

from skimage.transform import resize
from sklearn.model_selection import StratifiedKFold, GridSearchCV

import glob
import tensorflow as tf

from sklearn.metrics import confusion_matrix
tf.logging.set_verbosity(tf.logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size2, strides, padding):
  
    output = layers.Conv2D(filters=dim_capsule*n_channels, kernel_size=kernel_size2, strides=strides, padding=padding,
                           name='primarycap_conv1d1')(inputs)
    output_BN = layers.BatchNormalization()(output)
    output_Act = layers.Activation('relu')(output_BN)
    
    
    #conv2_max = Lambda(lambda x: K.max(x, axis=-1, keepdims=True))(output)
    #conv2_mean = Lambda(lambda x: K.mean(x, axis=-1, keepdims=True))(output)
    #conv2_concat = layers.Concatenate(axis=-1)([conv2_max, conv2_mean])
    
    #conv2_sig = layers.Activation('sigmoid')(conv2_max)
    
    #conv2_fil =layers.Conv1D(filters=1, kernel_size=1,strides=1, padding='same', name='conv1_fil')(conv2_sig)
    
    #conv2_sig2 = layers.Activation('sigmoid')(conv2_fil)
    #mult = output*conv2_sig2
    
    #conv2_sig2 = layers.Activation('sigmoid')(output)
   
    #cc = layers.Multiply()([conv2_sig2, conv2_max])
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output_Act)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)

In [3]:
def capsule(x, y,  ftnums1, ftnums2, ksize1, ksize2, strs, cats_nums, size, dim_caps):
    conv1 =Conv2D(filters=ftnums1, 
                  kernel_size=ksize1,strides=strs, padding='valid', 
                  activation='relu', name='conv1')(x)
    
    conv1_BN = layers.BatchNormalization()(conv1)
    conv1_Act = layers.Activation('relu')(conv1_BN)

    primarycaps = PrimaryCap(conv1_Act, 
                             dim_capsule=8, n_channels=int(ftnums2/8), 
                             kernel_size2=ksize2, strides=strs, padding='valid')

    digitcaps = CapsuleLayer(num_capsule=cats_nums, dim_capsule=dim_caps, num_routing=3,
                                 name='digitcaps')(primarycaps)

    out_caps = Length(name='capsnet')(digitcaps)

    masked_by_y = Mask()([digitcaps, y])
    masked = Mask()(digitcaps)

    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(512, activation='relu', input_dim=dim_caps*cats_nums))
    decoder.add(layers.Dense(1024, activation='relu'))
    decoder.add(layers.Dense(size*size, activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=(size, size, 1), name='out_recon'))
    
    return models, out_caps, decoder(masked_by_y)

In [4]:
images = np.load('/home/yi/forest/hardness/np/pressure_2.npy')
finger1 = images[:,:,:24]
finger2 = images[:,:,24:48]
finger3 = images[:,:,48:]

fg1 = finger1[:,:].reshape((780, 72, 24))
fg2 = finger2[:,:].reshape((780, 72, 24))
fg3 = finger3[:,:].reshape((780, 72, 24))
X = np.concatenate((fg1, fg3, fg2), axis=-1)
X = np.expand_dims(X, axis=-1)

cats_nums = 13
y_ohot = np.repeat(np.arange(cats_nums), 60)
np.random.seed(42)
shuffle_indices = np.random.permutation(np.arange(len(y_ohot)))

x = X[shuffle_indices]
y = y_ohot[shuffle_indices]

In [5]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=4)

In [6]:
x_input = Input(shape=[72, 72, 1])
y_input = layers.Input(shape=(cats_nums,))
K.set_learning_phase(1)
acc_per_fold = []
train_accur_per_fold = []
test_accur_per_fold = []
cm_data_per_fold = []
pred_per_fold = []
y_test_per_fold = []
recon_per_fold = []
fold_no = 1

for train, test in kfold.split(X, y):
    
    y_train = to_categorical(y[train], cats_nums)
    y_test = to_categorical(y[test], cats_nums)
    x_train = x[train]
    x_test = x[test]
    models, out_caps, decode_mask = capsule(x_input,y_input, 32, 64, 6, 6,  2, cats_nums, 72, 16)
    Cap = models.Model([x_input, y_input], [out_caps, decode_mask])


    #lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: 0.001 * (0.99 ** epoch))

    #early_stop = callbacks.EarlyStopping(monitor='val_accuracy', mode='max', min_delta=1)
    # compile the model
    Cap.compile(optimizer=optimizers.Adam(lr=0.001),
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.392],
                  metrics={'capsnet': 'accuracy'})

    history = Cap.fit([x_train, y_train],[y_train, x_train], batch_size=30, epochs=10,
              validation_data=[[x_test, y_test], [y_test, x_test]])
    
    train_accur_per_fold.append(history.history["capsnet_acc"])
    test_accur_per_fold.append(history.history["val_capsnet_acc"])
   
    scores = Cap.evaluate([x_test, y_test], [y_test, x_test], verbose=1)
    print(f'Score for fold {fold_no}: {Cap.metrics_names[3]} of {scores[3]}; {Cap.metrics_names[3]} of {scores[3]*100}%')
    acc_per_fold.append(scores[3] * 100)

    pred = Cap.predict([x_test, y_test])
    cm_data = confusion_matrix(y_test.argmax(axis=1), pred[0].argmax(axis=1))
    pred_per_fold.append(pred[0])
    recon_per_fold.append(pred[1])
    y_test_per_fold.append(y[test])
    cm_data_per_fold.append(cm_data)
    fold_no = fold_no + 1
    
train_acc = np.mean(train_accur_per_fold, axis = 0)
test_acc = np.mean(test_accur_per_fold, axis = 0)
cm_data = np.sum(cm_data_per_fold, axis=0)

acc = np.mean(np.array(acc_per_fold))
name = "shape2_" + "Cap_exp1"
path = "/home/yi/forest/Capsule/data/"
np.save(path + 'pred/'+name + '.npy', np.array(pred_per_fold))
np.save(path + 'y_test/'+name + '.npy', np.array(y_test_per_fold))
np.save(path + 'recon/'+name + '.npy', recon_per_fold)
np.save(path + 'cm_data/'+name + '.npy', cm_data)
np.save(path + 'final_acc/'+name + '.npy', np.array(acc_per_fold))
CM(cm_data, name)


Train on 702 samples, validate on 78 samples
Epoch 1/10
702/702 [==============================] - 6s 9ms/step - loss: 0.4810 - capsnet_loss: 0.4307 - decoder_loss: 0.1283 - capsnet_acc: 0.4473 - val_loss: 0.2584 - val_capsnet_loss: 0.2529 - val_decoder_loss: 0.0140 - val_capsnet_acc: 0.6923
Epoch 2/10
702/702 [==============================] - 5s 7ms/step - loss: 0.1918 - capsnet_loss: 0.1858 - decoder_loss: 0.0152 - capsnet_acc: 0.8575 - val_loss: 0.1279 - val_capsnet_loss: 0.1220 - val_decoder_loss: 0.0150 - val_capsnet_acc: 0.9231
Epoch 3/10
702/702 [==============================] - 5s 7ms/step - loss: 0.0952 - capsnet_loss: 0.0894 - decoder_loss: 0.0148 - capsnet_acc: 0.9601 - val_loss: 0.0734 - val_capsnet_loss: 0.0679 - val_decoder_loss: 0.0139 - val_capsnet_acc: 0.9744
Epoch 4/10
702/702 [==============================] - 5s 7ms/step - loss: 0.0584 - capsnet_loss: 0.0528 - decoder_loss: 0.0144 - capsnet_acc: 0.9886 - val_loss: 0.0605 - val_capsnet_loss: 0.0551 - val_decoder_lo

KeyboardInterrupt: 

In [ ]:
acc